In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Default Parameters
versions = 2  # Number of versions including control

# Global variables to store the results
obs_conv_imprvs = None
true_conv_imprvs = None
std_obs_conv_imprvs = None
x_axis_visitors = None

# Function to Run Simulation
def run_simulation(button=None):
    global obs_conv_imprvs, true_conv_imprvs, std_obs_conv_imprvs, x_axis_visitors
    
    with output:
        # Clearing previous output
        clear_output(wait=True)
        
        # Getting values from widgets
        base_conv = base_conv_widget.value / 100  # converting to proportion
        improvement = improvement_widget.value / 100  # converting to proportion
        aa_test = aa_test_widget.value
        new_vis_per_iter = new_vis_per_iter_widget.value
        set_iter = set_iter_widget.value
        num_sims = num_sims_widget.value
        random_seed = random_seed_widget.value if random_seed_widget.value != 0 else None

        # Setting the maximum value for simulation slider
        sim_slider.max = num_sims - 1

        # Setting random seed
        np.random.seed(random_seed)

        # Code for simulation and visualization based on user-set parameters
        conv_rates = np.array([base_conv, base_conv * (1 + improvement_widget.value / 100)])  # converting to proportion
        if aa_test:
            conv_rates[1:] = conv_rates[0]

        true_conv_imprvs = ((conv_rates[1:] - conv_rates[0]) / conv_rates[0]) * 100

        orders = np.zeros((set_iter+1, versions*num_sims))
        visitor_counts = np.zeros((set_iter+1, versions*num_sims))
        for iter in range(set_iter):
            visitors_control = np.random.rand(new_vis_per_iter, num_sims) < conv_rates[0]
            visitors_treatment = np.random.rand(new_vis_per_iter, num_sims) < conv_rates[1]
            orders[iter+1, :num_sims] = orders[iter, :num_sims] + np.sum(visitors_control, axis=0)
            orders[iter+1, num_sims:] = orders[iter, num_sims:] + np.sum(visitors_treatment, axis=0)
            visitor_counts[iter+1, :num_sims] = visitor_counts[iter, :num_sims] + new_vis_per_iter
            visitor_counts[iter+1, num_sims:] = visitor_counts[iter, num_sims:] + new_vis_per_iter

        visitor_counts = np.maximum(visitor_counts, 1)

        obs_conv_rates = orders / visitor_counts
        obs_conv_imprvs = np.zeros((set_iter+1, (versions-1)*num_sims))
        for v in range(1, versions):
            obs_conv_imprvs[:, (v-1)*num_sims:v*num_sims] = (
                ((obs_conv_rates[:, v*num_sims:(v+1)*num_sims] - obs_conv_rates[:, :num_sims]) /
                np.maximum(obs_conv_rates[:, :num_sims], 1e-8)) * 100
            )

        avg_obs_conv_imprvs = np.nanmean(obs_conv_imprvs, axis=1)
        std_obs_conv_imprvs = np.nanstd(obs_conv_imprvs, axis=1)
        x_axis_visitors = np.arange(set_iter+1) * new_vis_per_iter

        update_plot()

# Function to Update Plot
def update_plot(change=None):
    with output:
        clear_output(wait=True)
        sim_num = sim_slider.value
        plt.figure(figsize=(10, 6), dpi=150)
        plt.plot(x_axis_visitors, obs_conv_imprvs[:, sim_num], label=f"Simulation {sim_num+1}")
        plt.fill_between(x_axis_visitors, 
                         -std_obs_conv_imprvs + true_conv_imprvs, 
                         std_obs_conv_imprvs + true_conv_imprvs, 
                         color='gray', alpha=0.5, label="1x Standard Deviation (~68% of simulations)")
        plt.fill_between(x_axis_visitors, 
                         -2*std_obs_conv_imprvs + true_conv_imprvs, 
                         2*std_obs_conv_imprvs + true_conv_imprvs, 
                         color='gray', alpha=0.2, label="2x Standard Deviation (~95% of simulations)")
        plt.axhline(y=true_conv_imprvs, color='r', linestyle='-', label=f"True Conversion Improvement ({true_conv_imprvs[0]:.2f}%)")
        plt.title("Observed Conversion Improvements Over Time")
        plt.xlabel("Number of Visitors")
        plt.ylabel("Conversion Improvement (%)")
        plt.legend()
        plt.show()

# Widgets for user input parameters
base_conv_widget = widgets.FloatSlider(min=1, max=100, step=1, value=50, description='Base Conv (%)')
improvement_widget = widgets.FloatSlider(min=-20, max=20, step=0.1, value=1, description='Improvement (%)')
aa_test_widget = widgets.Checkbox(value=False, description='AA Test')
new_vis_per_iter_widget = widgets.IntSlider(min=100, max=10000, step=100, value=1000, description='New Visitors/Iter')
set_iter_widget = widgets.IntSlider(min=10, max=1000, step=10, value=100, description='Set Iterations')
num_sims_widget = widgets.IntSlider(min=10, max=1000, step=10, value=50, description='Num Simulations')
random_seed_widget = widgets.IntText(value=None, description='Random Seed', allow_none=True)

# Adding a button to run the simulation
run_button = widgets.Button(description="Run Simulations")
run_button.on_click(run_simulation)

# Adding a slider to choose the simulation number
sim_slider = widgets.IntSlider(min=0, max=num_sims_widget.value-1, step=1, value=0, description='Show Simulation:')
sim_slider.observe(update_plot, names='value')

# Displaying the widgets
display(base_conv_widget, improvement_widget, aa_test_widget, new_vis_per_iter_widget, 
        set_iter_widget, num_sims_widget, random_seed_widget, run_button, sim_slider)

# Output widget to display the plot and values
output = widgets.Output()
display(output)

# Calling the function once to display the initial plot
run_simulation()

FloatSlider(value=50.0, description='Base Conv (%)', min=1.0, step=1.0)

FloatSlider(value=1.0, description='Improvement (%)', max=20.0, min=-20.0)

Checkbox(value=False, description='AA Test')

IntSlider(value=1000, description='New Visitors/Iter', max=10000, min=100, step=100)

IntSlider(value=100, description='Set Iterations', max=1000, min=10, step=10)

IntSlider(value=50, description='Num Simulations', max=1000, min=10, step=10)

IntText(value=0, description='Random Seed')

Button(description='Run Simulations', style=ButtonStyle())

IntSlider(value=0, description='Show Simulation:', max=49)

Output()